In [1]:
#!pip install tensorflow

### Paso 1: Importar librerías necesarias

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Paso 2: Preparación y carga de datos

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2)  # Usamos el 20% de los datos para validación

train_generator = train_datagen.flow_from_directory(
    'data/',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training')  # Conjunto de entrenamiento

validation_generator = train_datagen.flow_from_directory(
    'data/',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation')  # Conjunto de validación


### Paso 3: Construcción del modelo

In [4]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')  # Suponiendo que hay 6 clases (0-5)
])

### Paso 4: Compilar el modelo

In [5]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Paso 5: Entrenamiento del modelo

In [6]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=25)


### Paso 6: Evaluación del modelo

In [7]:
# Evaluar el modelo en el conjunto de validación
eval_result = model.evaluate(validation_generator)
print(f'Loss: {eval_result[0]}, Accuracy: {eval_result[1]}')

### Paso 7: Guardar el modelo

In [8]:
# Guardar el modelo en formato HDF5
model.save('hand_number_model.h5')

# Para guardar en formato SavedModel, que es más completo, usarías:
# model.save('hand_number_model', save_format='tf')